In [73]:
import os
%run funciones.py
import json
import pandas as pd
import numpy as np
from funciones import *
from keras.models import load_model

In [74]:
def predecir1(x, model, scaler, conv):
    # Calcular predicción escalada en el rango de -1 a 1
    y_pred_s = model.predict(x,verbose=1)
    
    if conv:
       y_pred_s = y_pred_s[:, 0, 0, 0].reshape((-1, 1))

    # Llevar la predicción a la escala original
    y_pred = scaler.inverse_transform(y_pred_s)

    return y_pred.flatten()

In [75]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def RMSE_MSE_VAL_SEC(Real_vec, Pred_vec):
    vector_real = Real_vec.flatten()
    vector_predicho = Pred_vec

    # Calcular las métricas RSME - MSE - MAE
    rmse = np.sqrt(mean_squared_error(vector_real, vector_predicho))
    mae = mean_absolute_error(vector_real, vector_predicho)
    mape = mean_absolute_percentage_error(vector_real, vector_predicho)

    # Calcular el RMSE
    return rmse, mae, mape

In [76]:
pruebas = pd.read_csv("..\data\pruebas.csv")

stop = list(pruebas['name stops'])
real = list(pruebas['real'])
link = list(zip(pruebas['ini_link'], pruebas['end_link']))
time = list(zip(pruebas['hora'], pruebas['minuto']))

c = ['Nro','Stops','Link', 'Time', 'Real', 'Mean_Set', 'err_mean_set', 'input_set', 'err_input_set', 'input_concat', 'err_input_concat', 'model']
preds = pd.DataFrame(columns=c)

for file in os.listdir('../models/'):
    model_path = f'../models/{file}'
    json_path = f'../info_models/{file[:-2]}json'

    # cargar archivos
    info_model = json.load(open(json_path))
    model = load_model(model_path, compile=False)

    # ingresar parametros
    div_split = info_model['div_split']
    input_out_length = (info_model['INPUT_LENGTH'], info_model['OUTPUT_LENGTH'])  
    SCALER = info_model['scaler']
    CONV = info_model['isCONV']
    range_links = tuple(info_model['range_links'])
    path_dataset = "../data/"+info_model['name_dataset']
    input_shape = (1, ) + model.input_shape[1:]
    print(f"{file}")
    print(f"Parametros de prediccion: {div_split}, {input_out_length}, {SCALER}, CONV:{CONV}, {range_links}, {path_dataset}, {input_shape}")

    # leer dataset
    cols = ['id_linkref', 'horas', 'minute', 'travel_time']
    origin_dataset = pd.read_csv(path_dataset, sep=',', usecols=cols)
    dataset = origin_dataset[cols]

    # delimitar links
    if range_links != (0, 82):
        dataset =  dataset[(dataset['id_linkref']>= range_links[0]) & (dataset['id_linkref']<=range_links[1])]
    print(f"Links del dataset desde {range_links[0]} - {range_links[1]} = {len(dataset['id_linkref'].unique())} links")

    # preparar dataset para la prediccion segun el modelo
    data, scaler = cargar_dataset(dataset, div_split, input_out_length, SCALER, CONV)

    # cacular RMSE datos no escalados
    # y_tr_pred = predecir(x_tr_s, model, SCALER)
    #y_ts_pred = predecir1(data['x_ts_s'], model, scaler, conv=CONV)
    #rr = RMSE_MSE_VAL_SEC(data['y_ts'], y_ts_pred)
    #print(rr)
    # y_vl_pred = predecir(x_vl_s, model, SCALER)

    dt_input = data['ts'].reset_index()
    for i in range(len(real)):
        # encontrar input: retorna indices de los samples que coniden como input
        res = find_input(dt_input, stop=link[i][0], time=time[i], input_size=input_out_length[0])

        # escoger un indice aleatorio para pa prediccion
        #if len(res)==0:
        #    print(f'sample {i} sin input coincidente')
        #    continue
        tmp = np.random.choice(len(res))

        # predecir el tiempo de viaje
        mean, y_of_set, y_of_pred = predict_travel_time(
            n_links = link[i][1] - link[i][0],
            shape_input= input_shape,
            id_sample= res[tmp],
            data_in_s= data['x_ts_s'],
            data_in = data['x_ts'],
            modelo = model,
            scaler=scaler,
            conv=CONV
        )

        # calcular tiempo de viaje, suma de todas las predicciones en cada link
        p_mean = round(sum(mean),0) # promedio, para verificar en que se basa las predicciones
        p_set = round(sum(y_of_set), 0) # usamos input un grupo de muestras del dataset test
        p_concat = round(sum(y_of_pred), 0) # concadenamos las predicciones al siguiente input

        # calcular el error de cada forma de prediccion
        err_p_mean = real[i] - p_mean
        err_p_set = real[i] - p_set
        err_p_concat = real[i] - p_concat 

        # agregar al df pred
        preds.loc[len(preds)] = [i+1, stop[i], link[i], time[i], real[i], p_mean, err_p_mean, p_set, err_p_set, p_concat, err_p_concat, file[:-3]]

    print('=> Pruebas Completadas')
    print('='*120)

ConvLSTMEnco_Robus_e150b256_80e.h5
Parametros de prediccion: 80, (40, 1), Robust, CONV:True, (2, 81), ../data/travel_time_series_2col.csv, (1, 40, 4, 1, 1)
Links del dataset desde 2 - 81 = 80 links
=> Pruebas Completadas
ConvLSTMEnco_Robus_epo150_batch256-36.h5
Parametros de prediccion: 36, (36, 1), Robust, CONV:True, (14, 49), ../data/travel_time_series_2col.csv, (1, 36, 4, 1, 1)
Links del dataset desde 14 - 49 = 36 links
=> Pruebas Completadas
LSMT2_e100b256_Robust_in36-4_dt14-49.h5
Parametros de prediccion: 36, (36, 1), Robust, CONV:False, (14, 49), ../data/travel_time_series_2col.csv, (1, 36, 4)
Links del dataset desde 14 - 49 = 36 links
=> Pruebas Completadas
LSMT2_e100b256_Robust_in40-4_dt0-82.h5
Parametros de prediccion: 40, (40, 1), Robust, CONV:False, (0, 82), ../data/travel_time_series_2col.csv, (1, 40, 4)
Links del dataset desde 0 - 82 = 83 links
=> Pruebas Completadas


In [77]:
preds

,Nro,Stops,Link,Time,Real,Mean_Set,err_mean_set,input_set,err_input_set,input_concat,err_input_concat,model
0,1,Primer Paradero - Hospital,"(39, 45)","(10, 30)",379,426.0,-47.0,531.0,-152.0,531.0,-152.0,ConvLSTMEnco_Robus_e150b256_80e
1,2,Primer Paradero - UNSAAC,"(39, 46)","(10, 30)",457,423.0,34.0,632.0,-175.0,632.0,-175.0,ConvLSTMEnco_Robus_e150b256_80e
2,3,Primer Paradero - Hospital,"(39, 45)","(16, 45)",397,536.0,-139.0,450.0,-53.0,449.0,-52.0,ConvLSTMEnco_Robus_e150b256_80e
3,4,Primer Paradero - UNSAAC,"(39, 46)","(16, 45)",513,623.0,-110.0,556.0,-43.0,556.0,-43.0,ConvLSTMEnco_Robus_e150b256_80e
4,5,Control - Universidad Andina,"(14, 21)","(12, 45)",374,212.0,162.0,242.0,132.0,242.0,132.0,ConvLSTMEnco_Robus_e150b256_80e
5,6,Control - Universidad Andina,"(14, 21)","(13, 45)",397,320.0,77.0,243.0,154.0,243.0,154.0,ConvLSTMEnco_Robus_e150b256_80e
6,7,Control - Universidad Andina,"(14, 21)","(11, 0)",389,208.0,181.0,244.0,145.0,244.0,145.0,ConvLSTMEnco_Robus_e150b256_80e
7,8,Control - Universidad Andina,"(14, 21)","(14, 30)",371,211.0,160.0,245.0,126.0,245.0,126.0,ConvLSTMEnco_Robus_e150b256_80e
8,9,Universidad Andina - Puerta Universidad UNSAAC,"(22, 45)","(14, 30)",1271,1085.0,186.0,1166.0,105.0,1166.0,105.0,ConvLSTMEnco_Robus_e150b256_80e
9,10,Universidad Andina - Puerta Universidad UNSAAC,"(22, 45)","(10, 0)",1157,1119.0,38.0,1244.0,-87.0,1239.0,-82.0,ConvLSTMEnco_Robus_e150b256_80e


In [78]:
total_err_input_set = {}
total_err_input_concat = {}
for mod in preds['model'].unique():
    df_tmp = preds[preds['model'] == mod]
    total_err_input_set[mod] = abs(df_tmp['err_input_set']).sum()
    total_err_input_concat[mod] = abs(df_tmp['err_input_concat']).sum()

In [79]:
total_err_input_set

{'ConvLSTMEnco_Robus_e150b256_80e': 1341.0,
 'ConvLSTMEnco_Robus_epo150_batch256-36': 869.0,
 'LSMT2_e100b256_Robust_in36-4_dt14-49': 899.0,
 'LSMT2_e100b256_Robust_in40-4_dt0-82': 896.0}

In [80]:
total_err_input_concat

{'ConvLSTMEnco_Robus_e150b256_80e': 1335.0,
 'ConvLSTMEnco_Robus_epo150_batch256-36': 873.0,
 'LSMT2_e100b256_Robust_in36-4_dt14-49': 820.0,
 'LSMT2_e100b256_Robust_in40-4_dt0-82': 891.0}

In [81]:
# leer dataset busstop
df_stops = pd.read_csv('../data/busstops_ida.csv')

# df_stops.info()
def generateLinks(row):
    row['end_stop'] = df_linkref.loc[row.name+1, 'id']
    row['name_end_stop'] = df_linkref.loc[row.name+1, 'name']
    row['linkref'] = str(row['id']) + ':' + str(df_linkref.loc[row.name+1, 'id'])
    return row

df_linkref = df_stops.copy()
df_linkref = df_linkref.head(df_linkref.shape[0]-1).apply(generateLinks, axis=1)
df_linkref = df_linkref.rename(columns={'id':'init_stop'})
df_linkref.drop(columns=['number', 'radio', 'latitude', 'longitude', 'navigation'], inplace=True)
# df_linkref['end_stop'] = df_linkref['end_stop'].values.astype(str)
print(df_linkref.to_string())

    init_stop                       name  end_stop              name_end_stop  linkref
0           0                     inicio        36                primer stop     0:36
1          36                primer stop        37               segundo stop    36:37
2          37               segundo stop        38                   Ccollana    37:38
3          38                   Ccollana        39            Puente Huaccoto    38:39
4          39            Puente Huaccoto        40                      Kayra    39:40
5          40                      Kayra        41                   Forestal    40:41
6          41                   Forestal        42                Grifo Tapia    41:42
7          42                Grifo Tapia        43                      Posta    42:43
8          43                      Posta        44                  Romeritos    43:44
9          44                  Romeritos        45                    Tercero    44:45
10         45                    Tercero   

In [82]:
a = 1157
a/60,a%60

(19.283333333333335, 17)